In [1]:
import pandas as pd
import numpy as np

In [2]:
orders = pd.read_csv('../data/raw/olist_orders_dataset.csv')
order_items = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
customers = pd.read_csv('../data/raw/olist_customers_dataset.csv')
products = pd.read_csv('../data/raw/olist_products_dataset.csv')
products_category_translation = pd.read_csv('../data/raw/product_category_name_translation.csv')

In [3]:
orders.info()
orders.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [4]:
order_items.info()
order_items.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [5]:
customers.info()
customers.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [6]:
customers.duplicated().sum()

np.int64(0)

In [7]:
orders_delivered = orders[orders['order_status'] == 'delivered'].copy()

In [8]:
orders_delivered.isnull().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
dtype: int64

In [9]:
#orders_delivered['delivery_time_days'] = (
   # orders_delivered['order_delivered_customer_date'] -
    #orders_delivered['order_purchase_timestamp']
#).dt.days
#Erro aconteceu por conta das tabelas não terem sido convertidas em data

In [10]:
orders_delivered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96478 non-null  object
 1   customer_id                    96478 non-null  object
 2   order_status                   96478 non-null  object
 3   order_purchase_timestamp       96478 non-null  object
 4   order_approved_at              96464 non-null  object
 5   order_delivered_carrier_date   96476 non-null  object
 6   order_delivered_customer_date  96470 non-null  object
 7   order_estimated_delivery_date  96478 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


In [11]:
#Conversão de datas
date_cols = [                       #Seleciona as colunas que serão alteradas
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
]

for col in date_cols:                       #Aqui percorremos cada coluna de data e garantimos o tipo correto,
    orders_delivered[col] =pd.to_datetime(  #usando errors='coerce' para evitar falhas e transformar valores inválidos em NaT
        orders_delivered[col],
        errors='coerce'
    )

In [12]:
orders_delivered[date_cols].dtypes #Validação dos tipos de dados

order_purchase_timestamp         datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [13]:
orders_delivered['delivery_time_days'] = (
    orders_delivered['order_delivered_customer_date'] -
    orders_delivered['order_purchase_timestamp']
).dt.days
#Tempo médio de entrega

In [14]:
orders_delivered['delivery_delay'] = (
    orders_delivered['order_delivered_customer_date'] >
    orders_delivered['order_estimated_delivery_date']
)
#Atraso vs prazo

In [15]:
orders_delivered [['delivery_time_days','delivery_delay']].head(10)

,delivery_time_days,delivery_delay
0,8.0,False
1,13.0,False
2,9.0,False
3,13.0,False
4,2.0,False
5,16.0,False
7,9.0,False
8,9.0,False
9,18.0,False
10,12.0,False


In [16]:
orders_delivered['delivery_time_days'].describe()

count    96470.000000
mean        12.093604
std          9.551380
min          0.000000
25%          6.000000
50%         10.000000
75%         15.000000
max        209.000000
Name: delivery_time_days, dtype: float64

In [17]:
orders_delivered['delivery_delay'].value_counts()
#Pedidos que atrasaram

delivery_delay
False    88652
True      7826
Name: count, dtype: int64

In [18]:
orders_delivered['delivery_delay'].mean()
#Percentual de atraso

np.float64(0.08111693857667032)

In [19]:
#Construção da Fato de Vendas
fato_vendas = orders_delivered.merge(
    order_items,
    on='order_id',
    how='inner'
)
#Junta pedidos entregues com itens vendidos
#inner garante que só entram itens de pedidos válidos
#cada linha vira 1 item vendido

In [20]:
fato_vendas.shape

(110197, 16)

In [21]:
fato_vendas.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,False,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,False,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,False,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,False,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,False,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [22]:
#Join com Clientes
fato_vendas = fato_vendas.merge(
    customers,
    on='customer_id',
    how='left'
)

In [23]:
#Join com produtos
fato_vendas = fato_vendas.merge(
    products,
    on='product_id',
    how='left'
)

In [24]:
fato_vendas = fato_vendas.merge(
    products_category_translation,
    on='product_category_name',
    how='left'
)

In [25]:
fato_vendas['purchase_year_month'] = (
    fato_vendas['order_purchase_timestamp']
    .dt.to_period('M')
    .astype(str)
)
fato_vendas[['order_purchase_timestamp','purchase_year_month']].head()

,order_purchase_timestamp,purchase_year_month
0,2017-10-02 10:56:33,2017-10
1,2018-07-24 20:41:37,2018-07
2,2018-08-08 08:38:49,2018-08
3,2017-11-18 19:28:06,2017-11
4,2018-02-13 21:18:39,2018-02


In [26]:
fato_vendas.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,purchase_year_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,False,...,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,False,...,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,2018-07
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,False,...,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto,2018-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,False,...,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,pet_shop,2017-11
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,False,...,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,stationery,2018-02


In [27]:
#Criação de métricas financeiras
fato_vendas['receita_item'] = fato_vendas['price'] + fato_vendas['freight_value']

In [28]:
fato_vendas[['price', 'freight_value', 'receita_item']].describe()

,price,freight_value,receita_item
count,110197.000000,110197.000000,110197.000000
mean,119.980563,19.948598,139.929161
std,182.299446,15.698136,189.319151
min,0.850000,0.000000,6.080000
25%,39.900000,13.080000,55.180000
50%,74.900000,16.260000,92.130000
75%,134.170000,21.150000,157.510000
max,6735.000000,409.680000,6929.310000


In [29]:
fato_vendas.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'delivery_time_days', 'delivery_delay', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'purchase_year_month', 'receita_item'],
      dtype='object')

In [30]:
cols_to_drop = [
    'customer_zip_code_prefix',
    'order_approved_at',
    'order_delivered_carrier_date'
]

fato_vendas.drop(columns=cols_to_drop, inplace=True)

In [31]:
fato_vendas.to_csv('../data/processed/fato_vendas.csv',
                   index=False
                   )

In [32]:
import sqlite3

In [33]:
conn = sqlite3.connect('../data/processed/analytics.db')

In [34]:
fato_vendas.to_sql(
    'fato_vendas',
    conn,
    if_exists='replace',
    index=False
)

110197

In [37]:
fato_vendas.head(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay,order_item_id,product_id,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,purchase_year_month,receita_item
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,8.0,False,1,87285b34884572647811a353c7ac498a,...,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,2017-10,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,13.0,False,1,595fac2a385ac33a80bd5114aec74eb8,...,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,2018-07,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,9.0,False,1,aa4383b373c6aca5d8797843e5594415,...,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto,2018-08,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,13.0,False,1,d0b61bfb1de832b15ba9d266ca96e5b0,...,59.0,468.0,3.0,450.0,30.0,10.0,20.0,pet_shop,2017-11,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,2.0,False,1,65266b2da20d04dbe00c5c2d3bb7859e,...,38.0,316.0,4.0,250.0,51.0,15.0,15.0,stationery,2018-02,28.62
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-26 10:57:55,2017-08-01,16.0,False,1,060cb19345d90064d1015407193c233d,...,49.0,608.0,1.0,7150.0,65.0,10.0,65.0,auto,2017-07,175.26
6,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-26 12:55:51,2017-06-07,9.0,False,1,4520766ec412348b8d4caa5e8a18c464,...,59.0,956.0,1.0,50.0,16.0,16.0,17.0,auto,2017-05,75.16
7,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-02-02 14:08:10,2017-03-06,9.0,False,1,ac1789e492dcd698c5c10b97a671243a,...,41.0,432.0,2.0,300.0,35.0,35.0,15.0,furniture_decor,2017-01,35.95
8,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-08-16 17:14:30,2017-08-23,18.0,False,1,9a78fb9862b10749a117f7fc3c31f051,...,45.0,527.0,1.0,9750.0,42.0,41.0,42.0,office_furniture,2017-07,169.76
9,e6ce16cb79ec1d90b1da9085a6118aeb,494dded5b201313c64ed7f100595b95c,delivered,2017-05-16 19:41:10,2017-05-29 11:18:31,2017-06-07,12.0,False,1,08574b074924071f4e201e151b152b4e,...,36.0,450.0,1.0,9000.0,42.0,12.0,39.0,garden_tools,2017-05,129.53


In [71]:
query = """ 
SELECT
    customer_state,
    SUM(receita_item) AS receita_estado,
    SUM (receita_item) / COUNT(DISTINCT order_id) as ticket_medio_estado
FROM fato_vendas
GROUP BY customer_state
ORDER BY ticket_medio_estado DESC
"""

In [72]:
pd.read_sql(query, conn)

,customer_state,receita_estado,ticket_medio_estado
0,PB,137838.55,266.612282
1,AC,19575.33,244.691625
2,AP,16141.81,240.922537
3,AL,94172.49,237.210302
4,RO,56966.00,234.427984
5,PA,212023.57,224.126395
6,PI,105178.19,220.962584
7,RR,9039.52,220.476098
8,TO,60007.37,219.005000
9,RN,100714.78,212.478439
